### Spark notebook ###

This notebook will only work in a Jupyter notebook or Jupyter lab session running on the cluster master node in the cloud.

Follow the instructions on the computing resources page to start a cluster and open this notebook.

**Steps**

1. Connect to the Windows server using Windows App.
2. Connect to Kubernetes.
3. Start Jupyter and open this notebook from Jupyter in order to connect to Spark.

In [ ]:
# Run this cell to import pyspark and to define start_spark() and stop_spark()

import findspark

findspark.init()

import getpass
import pandas
import pyspark
import random
import re

from IPython.display import display, HTML
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Constants used to interact with Azure Blob Storage using the hdfs command or Spark

global username

username = re.sub('@.*', '', getpass.getuser())

# Functions used below

def dict_to_html(d):
    """Convert a Python dictionary into a two column table for display."""
    html = []
    html.append(f'<table width="100%" style="width:100%; font-family: monospace;">')
    for k, v in d.items():
        html.append(f'<tr><td style="text-align:left;">{k}</td><td>{v}</td></tr>')
    html.append(f'</table>')
    return ''.join(html)

def show_as_html(df, n=20):
    """Leverage existing pandas jupyter integration to show a spark dataframe as html."""
    display(df.limit(n).toPandas())

def display_spark():
    """Display the status of the active Spark session if one is currently running."""
    if 'spark' in globals() and 'sc' in globals():
        name = sc.getConf().get("spark.app.name")
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:green">active</span></b>, look for <code>{name}</code> under the running applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://localhost:{sc.uiWebUrl.split(":")[-1]}" target="_blank">Spark Application UI</a></li>',
            f'</ul>',
            f'<p><b>Config</b></p>',
            dict_to_html(dict(sc.getConf().getAll())),
            f'<p><b>Notes</b></p>',
            f'<ul>',
            f'<li>The spark session <code>spark</code> and spark context <code>sc</code> global variables have been defined by <code>start_spark()</code>.</li>',
            f'<li>Please run <code>stop_spark()</code> before closing the notebook or restarting the kernel or kill <code>{name}</code> by hand using the link in the Spark UI.</li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))
    else:
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:red">stopped</span></b>, confirm that <code>{username} (notebook)</code> is under the completed applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))

In [ ]:
d# Spark imports

from pyspark.sql import Row, DataFrame, Window, functions as F
from pyspark.sql.types import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

%matplotlib inline

In [ ]:
# Color gradient function for correlation tables
def color_gradient(val):
    """Return color for value: blue for positive, red for negative, white for zero."""
    from matplotlib import colors
    cmap = colors.LinearSegmentedColormap.from_list('', ['red','white','blue'])
    norm = colors.Normalize(vmin=-1, vmax=1)
    rgb = cmap(norm(val))[:3]
    return f'background-color: rgb({int(rgb[0]*255)}, {int(rgb[1]*255)}, {int(rgb[2]*255)})'

In [ ]:
def stop_spark():
    """Stop the active Spark session and delete globals for SparkSession (spark) and SparkContext (sc)."""

    global spark
    global sc

    if 'spark' in globals() and 'sc' in globals():

        spark.stop()

        del spark
        del sc

    display_spark()


In [ ]:
print("_" * 35 + "HELPER / DIAGNOSTIC FUNCTIONS" + "_" * 35)

notebook_run_time = time.time()

def df_as_html(df, n: int = 5, right_align: bool = False, show_index: bool = False):
    """
    HTML preview via pandas with no truncation. If right_align=True,
    only numeric columns are right-justified; everything else is 
    explicitly left-aligned.
    """
    
    pdf = df.limit(n).toPandas()
    print("[INFO] Converting Spark → pandas for HTML display (rows:", len(pdf), ")")
    print("[INFO] right_align (numeric columns):", right_align)

    with pd.option_context(
        "display.max_colwidth", None,   
        "display.max_columns", None,    
        "display.width", None            
    ):
        styler = pdf.style if show_index else pdf.style.hide(axis="index")

        #   table alignment: left for both headers and cells
        styler = styler.set_table_styles(
            [
                {"selector": "th", "props": [("text-align", "left")]},
                {"selector": "td", "props": [("text-align", "left")]},
            ],
            overwrite=True,  # make this the baseline
        )
         
        if right_align:
            numeric_cols = list(pdf.select_dtypes(include=["number"]).columns)
            print("[INFO] Right-aligning numeric columns:", numeric_cols)
            if numeric_cols:
                styler = styler.set_properties(subset=numeric_cols,
                                               **{"text-align": "right"})
        display(styler)

def _normalise_dir(s: str) -> str:
    """
    Ensure trailing slash so we point to
    the dataset directory (not a file)
    """
    return s if s.endswith("/") else s + "/"

def ensure_dir(path: str) -> str:
    """
    ensures that path is a path 
    and not representing a file;
    add trailing slash if needed
    """
    if path is None:
        raise ValueError("Path is None")
    path = _normalise_dir(path)
#   print("ensure_dir -> ",path)
    return path
    
def show_df(df: DataFrame, n: int = 10, name: str = "", right_align: bool = False):
    """
    Print schema, 
    show an HTML sample,
    and row count.
    """
    bprint()
    print("name : ",name)
    df.printSchema()
    print("[check] sample:")
    df_as_html(df, n=n, right_align=right_align)
  
def write_parquet(df: DataFrame, dir_as_path: str, df_name:str = ""):    
    funct_time = time.time()
    path = _normalise_dir(dir_as_path)
    print(f"[file] write_parquet  : {path}")
    try:      
        show_df(df,df_name)
    except Exception as e:
        print("[cathch] sample failed:", e)
        os.system(f'hdfs dfs -rm -r -f "{path}"')   # idempotent cleanup
    df.write.mode("overwrite").format("parquet").save(path)
    os.system(f'hdfs dfs -ls -R "{path}"')
    funct_time = time.time() - funct_time 
    print(f"[time] write_parquet (min)   : {funct_time/60:5.2f}")
    print(f"[time] write_parquet (sec)   : {funct_time:5.2f}")
  
def has_parquet(dir_as_path: str) -> bool:
    path   = _normalise_dir( dir_as_path)
    marker = path + '_SUCCESS'
    #print("\n[check] dir_path:", dir_path)
    #print("\n[check] path    :", path)
    print("\n[check] marker  :", marker)
    rc = os.system(f'hdfs dfs -test -e "{marker}"')
    print("[check] rc:", rc, "->", ("exists" if rc == 0 else "missing"))
    return (rc == 0)
     
def _success_exists(target_dir: str) -> bool:
    """
    Check for the Hadoop/Spark _SUCCESS marker;  
    """
    jvm = spark._jvm
    hconf = spark._jsc.hadoopConfiguration()
    try:
        uri = jvm.java.net.URI(target_dir)
        fs = jvm.org.apache.hadoop.fs.FileSystem.get(uri, hconf)
        success = jvm.org.apache.hadoop.fs.Path(target_dir + "_SUCCESS")
        exists = fs.exists(success)
        print(f"[status] _SUCCESS check at: {target_dir}_SUCCESS -> {exists}")
        return bool(exists)
    except Exception as e:
        print(f"[status] _SUCCESS check failed ({e}); attempting read-probe …")
        try:
            spark.read.parquet(target_dir).limit(1).count()
            print(f"[dewstatus59] read-probe succeeded at: {target_dir}")
            return True
        except Exception as e2:
            print(f"[status] read-probe failed ({e2}); treating as not existing.")
            return False
 

def normalise_ids(df: DataFrame, col: str = "ID") -> DataFrame:
    """
    # Single source of truth for ID normalisation 
    Upper + trim + distinct on the given ID column.
    """
    print(f"[INFO] normalise_ids() on column: {col}")
    df.printSchema()
    df.show(20)
    return df.select(F.upper(F.trim(F.col(col))).alias("ID")).distinct()
    df.printSchema()
    df.show(20)
# hack 
_ids       = normalise_ids
canon_ids  = normalise_ids
_canon_ids = normalise_ids

def probe_univ 
def _count_unique_ids(df: DataFrame) -> int:
    return normalise_ids(df).count()

def pick_unfiltered_daily(preferred_path: str = None) -> DataFrame:
    """Return an unfiltered daily DF (~129k unique station IDs)."""
    cand_names = ["daily", "read_daily", "daily_df", "daily_all", "ghcnd_daily"]
    print("[INFO] Candidate DataFrames:", [n for n in cand_names if n in globals()])
    for name in cand_names:
        obj = globals().get(name)
        if isinstance(obj, DataFrame):
            try:
                n = normalise_ids(obj).count()
                print(f"[CHECK] {name} unique station IDs:", n)
                if n >= 120_000:
                    print(f"[INFO] Using {name} as the unfiltered daily.")
                    return obj
            except Exception as e:
                print(f"[WARN] Could not inspect {name}:", repr(e))
    if preferred_path:
        print(f"[INFO] Trying preferred_path: {preferred_path}")
        df = spark.read.parquet(str(preferred_path))
        n = normalise_ids(df).count()
        print("[CHECK] preferred_path unique station IDs:", n)
        if n >= 120_000:
            print("[INFO] Using preferred_path as the unfiltered daily.")
            return df
    for var in ["DAILY_READ_NAME","DAILY_WRITE_NAME","daily_read_name","daily_write_name","DAILY_NAME"]:
        if var in globals():
            path = globals()[var]
            try:
                print(f"[INFO] Trying {var} = {path}")
                df = spark.read.parquet(str(path))
                n = normalise_ids(df).count()
                print(f"[CHECK] {var} unique station IDs:", n)
                if n >= 120_000:
                    print(f"[INFO] Using {var} as the unfiltered daily.")
                    return df
            except Exception as e:
                print(f"[WARN] Could not read {var}:", repr(e))
    raise SystemExit("[FATAL] Could not find an unfiltered daily dataset (expected ~129k unique station IDs).")

def bprint(text: str="", l=50):
    n = len(text)
    n = abs(n - l)//2
    
    print("\n" + "_" * n + text + "_" * n)

# Back-compat aliases hack to account for non-disciplined naming un-convention
_ids       = normalise_ids
canon_ids  = normalise_ids
_canon_ids = normalise_ids

#print("[TEST] Using _canon_ids:", _canon_ids(stations).count())
#print("[TEST] Using canon_ids :", canon_ids(stations).count())
#print("[TEST] Using _ids      :", _ids(stations).count())

 
def _parse_ls_bytes(line): 
    parts = line.split()
    if len(parts) < 8:
        return None, None
    try:
        size = int(parts[4])
    except ValueError:
        return None, None
    return size, parts[-1]

def _parse_du_bytes(line):
    parts = line.split()
    if len(parts) < 2:
        return None, None
    try:
        size = int(parts[0])
    except ValueError:
        return None, None
    return size, parts[-1]

def du_bytes(path):
    lines = get_ipython().getoutput(f'hdfs dfs -du "{path}"')
    total = 0
    for ln in lines:
        parts = ln.split()
        if len(parts) >= 2:
            try:
                total += int(parts[0])
            except ValueError:
                pass
    return total
    
def benchmark_spark_distances(cities, radius_km=6368.6, repeats=3):
    """
    Uses Spark built-ins only. Measures full execution
    time by forcing an action.
    
    returns: dict with seconds for haversine/slc and
    row counts used.
    
    For the radius:
    
    The Earth is slightly flattened, so the geocentric 
    radius depends on latitude.  For context: 
    
    * equatorial radius = 6,378.137 km; 
    * polar radius      = 6,356.752 km 
    
    Across New Zealand’s latitudes (≈36–47°S), using the
    WGS-84 ellipsoid, you get roughly:

    Auckland (37°S):       ~6,370.4 km
    Christchurch (43.5°S): ~6,368.0 km
    Dunedin (45.9°S):      ~6,367.2 km
    __________________________________
    Wellington (41°S):     ~6,369.0 km
    mean                  ≈ 6,368.6 km
    """

    
    try:
        from pyspark.sql import SparkSession, functions as F, types as T
    except Exception:
        return None  # no Spark therefore save cannot run in vs code

    spark = SparkSession.getActiveSession()
    if spark is None:
        return None

    # build pairs once and cache
    schema = T.StructType([
        T.StructField("city", T.StringType(), False),
        T.StructField("lat",  T.DoubleType(), False),
        T.StructField("lon",  T.DoubleType(), False),
    ])
    df = spark.createDataFrame(cities, schema)
    a, b = df.alias("a"), df.alias("b")
    pairs = (a.join(b, F.col("a.city") < F.col("b.city"))
               .select(F.col("a.lat").alias("lat1"),
                       F.col("a.lon").alias("lon1"),
                       F.col("b.lat").alias("lat2"),
                       F.col("b.lon").alias("lon2"))
               .cache())
    _ = pairs.count()

    R = F.lit(float(radius_km))
    lat1 = F.radians(F.col("lat1")); lat2 = F.radians(F.col("lat2"))
    dlat = lat2 - lat1
    dlon = F.radians(F.col("lon2") - F.col("lon1"))

    # Haversine expr
    a_term = F.sin(dlat/2)**2 + F.cos(lat1)*F.cos(lat2)*F.sin(dlon/2)**2
    c_term = 2*F.atan2(F.sqrt(a_term), F.sqrt(1 - a_term))
    hav    = R * c_term

    # SLC expr
    cosv = F.sin(lat1)*F.sin(lat2) + F.cos(lat1)*F.cos(lat2)*F.cos(dlon)
    cosv = F.greatest(F.lit(-1.0), F.least(F.lit(1.0), cosv))
    slc = R * F.acos(cosv)

    # time Haversine
    t0 = perf_counter()
    for _ in range(repeats):
        _ = pairs.select(hav.alias("d")).agg(F.sum("d")).collect()
    t1 = perf_counter()

    # time SLC
    t2 = perf_counter()
    for _ in range(repeats):
        _ = pairs.select(slc.alias("d")).agg(F.sum("d")).collect()
    t3 = perf_counter()

    return {
        "spark_pairs": pairs.count(),
        "spark_repeats": repeats,
        "spark_haversine_sec": t1 - t0,
        "spark_slc_sec":       t3 - t2,
    }


In [ ]:
bprint("SECTION 1: ENVIRONMENT SETUP")
# supports: Assignment Setup — "Configure global variables and paths required for GHCN data processing"
# does: initializes runtime tracking, configures Azure storage paths, defines data locations,
#       establishes file paths, and sets debug flags for conditional processing

notebook_run_time = time.time()
val               = spark.range(1).select(F.date_format(F.current_timestamp(), 'yyyy.MM.dd HH:mm').alias('t')).first()['t']
bprint()
print(f"[time] current time           :  {val}")
bprint()


bprint("ENVIRONMENT") 
print("Spark       :", spark.version)
print("Python tuple:", sys.version_info[:3]) 
print("username    :", username)
print()

bprint("DEEBUG LOGICALS")
#FORCE_OVERWRITE = False  # False means that if the file exists then we wont re-write it 
#FORCE_OVERWRITE = True   # True means overwrite all resultant files

FORCE_REBUILD_ENRICHED  = False   #has_parquet(enriched_write_name)
FORCE_REBUILD_OVERLAP   = False    #has_parquet(overlap_write_name)
FORCE_REBUILD_PRECIP    = False    #has_parquet(precip_write_path)

FORCE_REBUILD_STATIONS  = False    #has_parquet(stations_write_name)
FORCE_REBUILD_INVENTORY = False    # has_parquet(inventory_write_name)
FORCE_REBUILD_STATES    = False    #has_parquet(states_write_name)
FORCE_REBUILD_COUNTRIES = False    #has_parquet(countries_write_name)
FORCE_REBUILD_INV_AGG   = False    # has_parquet(inv_agg_write_name)


print(f"[status] FORCE_REBUILD_ENRICHED  :", FORCE_REBUILD_ENRICHED)
print(f"[status] FORCE_REBUILD_INV_AGG   :", FORCE_REBUILD_INV_AGG)
print(f"[status] FORCE_REBUILD_STATIONS  :", FORCE_REBUILD_STATIONS)
print(f"[status] FORCE_REBUILD_INVENTORY :", FORCE_REBUILD_INVENTORY)
print(f"[status] FORCE_REBUILD_STATES    :", FORCE_REBUILD_STATES)
print(f"[status] FORCE_REBUILD_COUNTRIES :", FORCE_REBUILD_COUNTRIES)

print(f"[status] FORCE_REBUILD_OVERLAP   :", FORCE_REBUILD_OVERLAP)
print(f"[status] FORCE_REBUILD_PRECIP    :", FORCE_REBUILD_PRECIP)

bprint("SOURCE FOLDERS")
print()

azure_account_name        = "madsstorage002"
azure_data_container_name = "campus-data"
azure_user_container_name = "campus-user"
previous_year             = 2024  # full hear
most_recent_year          = 2025  # currently building


print("azure_account_name        :", azure_account_name)
print("azure_data_container_name :", azure_data_container_name)
print("azure_user_container_name :", azure_user_container_name)
print("previous_year             :", previous_year)
print("most_recent_year          :", most_recent_year)
print()

data_root      = f"wasbs://{azure_data_container_name}@{azure_account_name}.blob.core.windows.net/ghcnd/"
user_root      = f"wasbs://{azure_user_container_name}@{azure_account_name}.blob.core.windows.net/{username}/"
 
data_root      = ensure_dir(data_root)
user_root      = ensure_dir(user_root) 

print("data_root           :", data_root) 
print("user_root           :", user_root)
bprint()

daily_root     = ensure_dir(f"{data_root}daily/")

print("daily_root          :", daily_root)
print()

aux_root = "../auxiliary/"
aux_root = ensure_dir(aux_root)

reports_dir  = ensure_dir(f"{aux_root}reports/")
images_dir   = ensure_dir(f"{aux_root}images/")
figures_dir  = ensure_dir(f"{aux_root}figures/") 


print("aux_root    :", aux_root)
print("reports_dir :", reports_dir)
print("images_dir  :", images_dir)
print("figures_dir :", figures_dir)
print()



In [ ]:
# Run this cell before closing the notebook or kill your spark application by hand using the link in the Spark UI

stop_spark()